The aim of this notebook is to take the raw data and transform into a clean dataset set for modelling.

In [ ]:
import pandas as pd

In [6]:
order_numbers = pd.read_excel('data/data_task.xlsx', sheet_name='order_numbers')
transaction_data = pd.read_excel('data/data_task.xlsx', sheet_name='transaction_data')
reported_data = pd.read_excel('data/data_task.xlsx', sheet_name='reported_data')

# Clean the order numbers df
    - order df by date
    - ensure the order numbers value increase (create boolean column) 

In [10]:
order_numbers.shape

(856, 2)

In [7]:
order_numbers.head()

,date,order_number
0,2018-01-07,33841906
1,2018-01-22,34008921
2,2018-01-25,34397468
3,2018-02-06,34434432
4,2018-02-08,34579365


In [8]:
order_numbers.dtypes

date            datetime64[ns]
order_number             int64
dtype: object

In [15]:
def validate_order(df, date_col, order_col):
    # Ensure DataFrame is sorted by date
    df = df.sort_values(by=date_col).reset_index(drop=True)

    # Create Boolean column to check if order number is higher than the previous row
    df["order_increasing"] = df[order_col].diff() > 0  

    # Set the first row to True (as there's no previous row to compare)
    df.loc[0, "order_increasing"] = True 

    return df

In [16]:
orders1 = validate_order(order_numbers, 'date', 'order_number')
orders1

,date,order_number,order_increasing
0,2018-01-07,33841906,True
1,2018-01-22,34008921,True
2,2018-01-25,34397468,True
3,2018-02-06,34434432,True
4,2018-02-08,34579365,True
...,...,...,...
851,2022-12-26,89053562,True
852,2022-12-27,89078365,True
853,2022-12-28,89109007,True
854,2022-12-29,89139373,True


In [17]:
orders1.value_counts('order_increasing')

order_increasing
True     638
False    218
Name: count, dtype: int64

In [18]:
orders2 = orders1[orders1['order_increasing'] == True]
orders2.value_counts('order_increasing')

order_increasing
True    638
Name: count, dtype: int64